In [24]:
## imports for custom overlay
from pynq import Overlay
## Import for DMA alloc
from pynq import allocate
## import for video driver API support
from pynq.lib.video import *
# import for timing
import time
## CUSTOM OVERLAY
# 
hudson_overlay = Overlay('workingredshift3.bit')
# DMA vars
vdma = hudson_overlay.video.axi_vdma
dma = hudson_overlay.redDMA.axi_dma_0

In [36]:
# send input buffer and read the output
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
#     dma.sendchannel.transfer(shaped_in)
    dma.recvchannel.transfer(out_buffer)
#     dma.sendchannel.transfer(shaped_out)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [26]:
# Set HDMI in and out vars
hdmi_in = hudson_overlay.video.hdmi_in
hdmi_out = hudson_overlay.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)



In [27]:
hdmi_in.start()
hdmi_out.start()

In [28]:

# in_buffer = allocate(shape=frame.shape, dtype=np.uint8)
# out_buffer = allocate(shape=frame.shape, dtype=np.uint8)
in_buffer = allocate(shape=720*1280*3, dtype=np.uint8)
out_buffer = allocate(shape=720*1280*3, dtype=np.uint8)

# show on the screen what an in buffer frame looks like
shaped_in = hdmi_in.readframe()
in_buffer[:] = np.ndarray.flatten(shaped_in)
hdmi_out.writeframe(in_buffer)

In [29]:
# config axi lite access
resolution = hudson_overlay.redDMA.red_shift_0

In [30]:
# set resolution rows and cols via axi lite
resolution.write(0x10, 720)
resolution.write(0x18, 1280)

In [35]:
# point HDMI out to the out buffer
hdmi_out.writeframe(out_buffer)

numframes = 15
start = time.time()

# time 600 frames of processing
for _ in range(numframes):
    # read a frame, allocate a buffer to hold it
    in_buffer[:] = np.ndarray.flatten(hdmi_in.readframe())
    run_kernel()
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

    

Frames per second:  0.9638641043681159


1 frame per second when shifting the whole frame red in hardware

In [103]:
# read a frame, allocate a buffer to hold it
time.sleep(1)
in_buffer[:] = np.ndarray.flatten(hdmi_in.readframe())
run_kernel()

In [ ]:
print(shaped_in.shape)
for r in range(720):
    for c in range(1280):
        print("pixel", (r+1)*(c+1), in_buffer[(r+1)*(c+1)],in_buffer[(r+1)*(c+1)+1],in_buffer[(r+1)*(c+1)+2] )

In [ ]:
for r in range(720):
    for c in range(1280):
        print("pixel", (r+1)*(c+1), ":", out_buffer[(r+1)*(c+1)],out_buffer[(r+1)*(c+1)+1],out_buffer[(r+1)*(c+1)+2] )

In [37]:
hdmi_in.close()
hdmi_out.close()
